# AGI Development Code Ver 1

Ist real step toward AGI, purpose of this exercise is to create a POC working model of multi dimentional tokenisation UST its mechanics which will include 
1. Basic transformation to sounds phonics
2. conversion to vectors
3. joining to create 3D vectors into basic shapes like protein molecules
4. create nodes that associate relative information attached with words
5. mood dimentions
6. processing that to basic synthetic symbols
7. send querry
8. retreive information
9. process in a separate dimentional areas for dimention based resolved solution
10. combine all solutions through RAG and GAN analysis
11. get a new idea and go through this process again

## libraries 